In [ ]:
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
penguins_df = pd.read_csv('COMP472-A1-datasets/penguins.csv')
# print(penguins_df.head())

In [ ]:
# print csv file info
print(penguins_df.info()) 

In [ ]:
# label encoding string values to int
le = preprocessing.LabelEncoder()
penguins_df['species'] = le.fit_transform(penguins_df['species'])
penguins_df['island'] = le.fit_transform(penguins_df['island'])
penguins_df['sex'] = le.fit_transform(penguins_df['sex'])

# print(penguins_df.info()) 
# print(penguins_df.head())

In [ ]:
# converting the features into 1-hot vectors
# penguins_df = pd.get_dummies(penguins_df, columns=['species'], prefix='species', dtype='int64')
# penguins_df = pd.get_dummies(penguins_df, columns=['sex'], prefix='sex', dtype='int64')
# penguins_df = pd.get_dummies(penguins_df, columns=['island'], prefix='island', dtype='int64')

# print(penguins_df.info()) 
# print(penguins_df.head())

In [ ]:
# plot pie chart based on output class species
total = penguins_df['species'].count()

num_adelie = penguins_df[penguins_df['species'] == 0]['species'].count()
percent_adelie = num_adelie / total * 100

num_chinstrap = penguins_df[penguins_df['species'] == 1]['species'].count()
percent_chinstrap = num_chinstrap / total * 100

num_gentoo = penguins_df[penguins_df['species'] == 2]['species'].count()
percent_gentoo = num_gentoo / total * 100

species_percentages = [percent_adelie, percent_chinstrap, percent_gentoo]

plt.figure(figsize=(7, 7))
plt.pie(species_percentages, labels=['Adelie', 'Chinstrap', 'Gentoo'], autopct='%1.1f%%')
plt.title("Percentange of Each Penguin Species")
plt.savefig('penguins_species_pie_chart.png')

plt.show()